In [1]:
!pip install pyarrow
!pip install google-cloud-bigquery

In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [3]:
import psycopg2
import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account

In [4]:
conn = psycopg2.connect( host = 'localhost', port = 5432, database = 'AdventureWorks', user = 'postgres', password = 'admin' )

In [5]:
cur = conn.cursor()
table = pd.read_sql_query( 'Select * From  Sales.SalesOrderHeader, Sales.Creditcard, Sales.Salesorderdetail, Sales.Salesterritory WHERE creditcard.creditcardid = SalesOrderHeader.creditcardid and SalesOrderHeader.salesorderid = Salesorderdetail.salesorderid and Salesterritory.territoryid = SalesOrderHeader.territoryid and status = 5', conn )
cur.close()

In [6]:
conn.close()

In [7]:
table.dtypes

salesorderid                       int64
revisionnumber                     int64
orderdate                 datetime64[ns]
duedate                   datetime64[ns]
shipdate                  datetime64[ns]
status                             int64
onlineorderflag                     bool
purchaseordernumber               object
accountnumber                     object
customerid                         int64
salespersonid                    float64
territoryid                        int64
billtoaddressid                    int64
shiptoaddressid                    int64
shipmethodid                       int64
creditcardid                       int64
creditcardapprovalcode            object
currencyrateid                   float64
subtotal                         float64
taxamt                           float64
freight                          float64
totaldue                         float64
comment                           object
rowguid                           object
modifieddate    

In [8]:
table.head()

,salesorderid,revisionnumber,orderdate,duedate,shipdate,status,onlineorderflag,purchaseordernumber,accountnumber,customerid,...,territoryid,name,countryregioncode,group,salesytd,saleslastyear,costytd,costlastyear,rowguid,modifieddate
0,43659,8,2011-05-31,2011-06-12,2011-06-07,5,False,PO522145787,10-4020-000676,29825,...,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6dc4165a-5e4c-42d2-809d-4344e0ac75e7,2008-04-30
1,43659,8,2011-05-31,2011-06-12,2011-06-07,5,False,PO522145787,10-4020-000676,29825,...,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6dc4165a-5e4c-42d2-809d-4344e0ac75e7,2008-04-30
2,43659,8,2011-05-31,2011-06-12,2011-06-07,5,False,PO522145787,10-4020-000676,29825,...,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6dc4165a-5e4c-42d2-809d-4344e0ac75e7,2008-04-30
3,43659,8,2011-05-31,2011-06-12,2011-06-07,5,False,PO522145787,10-4020-000676,29825,...,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6dc4165a-5e4c-42d2-809d-4344e0ac75e7,2008-04-30
4,43659,8,2011-05-31,2011-06-12,2011-06-07,5,False,PO522145787,10-4020-000676,29825,...,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6dc4165a-5e4c-42d2-809d-4344e0ac75e7,2008-04-30


In [15]:
grouping = table.groupby( table[ 'orderdate' ].dt.date ).agg( { 'customerid': 'mean','revisionnumber':'count' } ).reset_index().rename( columns = { 'customerid': 'id_customer' } ).sort_values( by = 'orderdate' )

In [16]:
grouping

,orderdate,id_customer,revisionnumber
0,2011-05-31,29628.495798,357
1,2011-06-01,16571.250000,4
2,2011-06-02,21629.400000,5
3,2011-06-03,20559.500000,2
4,2011-06-04,19890.600000,5
...,...,...,...
1119,2014-06-26,20066.431034,58
1120,2014-06-27,18803.341463,82
1121,2014-06-28,18370.656250,64
1122,2014-06-29,19308.688525,61


In [17]:
credentials = service_account.Credentials.from_service_account_file( 
    'C:/Users/Rentadvisor/Downloads/taller-bigquery-48314721e491.json', 
    scopes = [ 'https://www.googleapis.com/auth/cloud-platform' ]
)

In [20]:
client = bigquery.Client( credentials = credentials, project = credentials.project_id )

In [23]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'orderdate', bigquery.enums.SqlTypeNames.DATE ),
        bigquery.SchemaField( 'id_customer', bigquery.enums.SqlTypeNames.FLOAT ),
        bigquery.SchemaField( 'revisionnumber', bigquery.enums.SqlTypeNames.INT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [24]:
job = client.load_table_from_dataframe(
    grouping, 'databases2.grouping', job_config = job_config
)
job.result()

In [25]:
table = client.get_table( 'databases2.grouping' )
print(
    'Loaded {} rows and {} columns to {}'.format(
        table.num_rows, len( table.schema ), 'databases2.grouping'
    )
)

Loaded 1124 rows and 3 columns to databases2.grouping
